In [ ]:
!pip install ultralytics

In [ ]:
# Get paths
import os
dir = r"E:\projects\_full_fledge\Kitchen-To-Ol\resources\annotation\Easie_Spoon\images"
image_paths = [os.path.join(dir, img) for img in os.listdir(dir) if not img.endswith(".txt")]
image_names = [img for img in os.listdir(dir)]

In [ ]:
# Mapping
idx_to_name = {
  0: "Spoon",
  1: "Fork",
  2: "Knife",
  3: "Tongs",
  4: "Bowl",
  5: "Pot",
}
name_to_idx = {v: k for k, v in idx_to_name.items()}

names_from_model = ["spoon", "fork", "knife", "bowl"]
names_from_model_to_idx = {
    "spoon": 0,
    "fork": 1,
    "knife": 2,
    "bowl": 4,
}

In [ ]:
# Auto annotation
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x.pt")  # load an official model

count = 0
for i, image_path in enumerate(image_paths):
    results = model(image_path, verbose = False)
    img_name = image_names[i]
    img_name = img_name.split(".")[0]
    
    # Access the results
    annotations = ""
    for result in results:
        xywh = result.boxes.xywh  # center-x, center-y, width, height
        xywhn = result.boxes.xywhn  # normalized
        xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
        xyxyn = result.boxes.xyxyn  # normalized
        names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
        confs = result.boxes.conf  # confidence score of each box
        for j, name in enumerate(names):
            bb = xywhn[j].detach().numpy()
            if name in list(names_from_model_to_idx.keys()):
                cls_idx = names_from_model_to_idx[name]
                cls_idx = 0
                annotations += f"{cls_idx} {bb[0]} {bb[1]} {bb[2]} {bb[3]}\n"

    if annotations != "":
        txt_path = os.path.join(dir, f"{img_name}.txt")
        try:
            with open(txt_path, "x") as fw:
                fw.write(annotations)
        except:
            with open(txt_path, "w") as fw:
                fw.write(annotations)
        print(f"Image {i} successfully annotated")
        count += 1
    else:
        print(f"Skipped Image {i}")
print(f"A total of {count} out of {len(image_paths)} images annotated")

### Visualization checking


In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import os
import shutil
import re

# --- CONFIGURATION ---
SOURCE_FOLDER = "." # The dir above
DEST_GOOD = os.path.join(SOURCE_FOLDER, "good")
DEST_BAD = os.path.join(SOURCE_FOLDER, "bad")
CLASSES = ["Spoon", "Fork", "Knife", "Tongs", "Bowl", "Pot"]

# --- HELPER: NATURAL SORT ---
# --- WINDOW RESIZING CONFIGURATION ---
MAX_WINDOW_DIM = 1000 # Set the maximum dimension (height or width) in pixels

def resize_to_max(img, max_dim=MAX_WINDOW_DIM):
    """Resizes the image if its longest side exceeds the max_dim."""
    h, w = img.shape[:2]
    
    # Check if the image needs resizing
    if max(h, w) <= max_dim:
        return img

    # Calculate the scale factor to maintain the aspect ratio
    scale = max_dim / max(h, w)
    new_w = int(w * scale)
    new_h = int(h * scale)

    # Use INTER_AREA for shrinking images, as it is efficient
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

# This ensures img1.jpg, img2.jpg, img10.jpg sort correctly
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', s)]

def draw_yolo_boxes(img, label_path):
    h, w, _ = img.shape
    if not os.path.exists(label_path):
        return img 

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5: continue
        
        cls_id = int(parts[0])
        cx, cy, bw, bh = map(float, parts[1:5])

        x1 = int((cx - bw / 2) * w)
        y1 = int((cy - bh / 2) * h)
        x2 = int((cx + bw / 2) * w)
        y2 = int((cy + bh / 2) * h)

        label_name = CLASSES[cls_id] if cls_id < len(CLASSES) else str(cls_id)
        
        # Green Box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Label with background for readability
        (w_text, h_text), _ = cv2.getTextSize(label_name, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(img, (x1, y1 - 20), (x1 + w_text, y1), (0, 255, 0), -1)
        cv2.putText(img, label_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    return img

def main():
    if not os.path.exists(DEST_GOOD): os.makedirs(DEST_GOOD)
    if not os.path.exists(DEST_BAD): os.makedirs(DEST_BAD)

    # 1. GET FILES AND SORT THEM
    all_files = os.listdir(SOURCE_FOLDER)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    
    # Apply Natural Sort (Crucial Fix)
    image_files.sort(key=natural_sort_key)
    
    total_images = len(image_files)
    print(f"Loaded {total_images} images. Sorted naturally.")
    cv2.namedWindow("YOLO Triage Helper", cv2.WINDOW_NORMAL)

    # 2. ITERATE
    # Using enumerate gives us a reliable index counter
    for i, filename in enumerate(image_files):
        img_path = os.path.join(SOURCE_FOLDER, filename)
        
        # Check if file still exists (in case you double-clicked or ran parallel scripts)
        if not os.path.exists(img_path):
            continue

        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(SOURCE_FOLDER, txt_filename)

        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping corrupt image: {filename}")
            continue

        display_img = img.copy()
        
        # 1. Draw boxes on the original image copy
        display_img = draw_yolo_boxes(display_img, txt_path)

        # 2. Add Progress Text
        label_text = f"Image {i+1}/{total_images}: {filename}"
        cv2.putText(display_img, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 3. Resize the image for screen display
        display_img = resize_to_max(display_img)
        
        # Show image
        cv2.imshow("YOLO Triage Helper", display_img)
        
        print(f"Processing {i+1}/{total_images}: {filename} ... ", end="")

        while True:
            key = cv2.waitKey(0) & 0xFF
            
            if key == ord('q'): # Quit
                print("Quitting.")
                cv2.destroyAllWindows()
                return
            
            elif key == ord('y'): # YES -> Good Folder
                print("ACCEPTED")
                shutil.move(img_path, os.path.join(DEST_GOOD, filename))
                if os.path.exists(txt_path):
                    shutil.move(txt_path, os.path.join(DEST_GOOD, txt_filename))
                break # Break inner while loop to go to next image

            elif key == ord('n') or key == 32: # NO (or Spacebar) -> Bad Folder
                print("REJECTED")
                shutil.move(img_path, os.path.join(DEST_BAD, filename))
                if os.path.exists(txt_path):
                    shutil.move(txt_path, os.path.join(DEST_BAD, txt_filename))
                break # Break inner while loop to go to next image

    cv2.destroyAllWindows()
    print("All done!")

if __name__ == "__main__":
    main()